In [15]:
# Connect to postgresql db
import psycopg2
from psycopg2 import OperationalError

# Database connection parameters
host = "localhost"  # or your database host
database = "thesis_match_db"
user = "user"
password = "password"
port = 5432

try:
    # Create connection
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    
    # Create cursor
    cursor = connection.cursor()
    
    # Test connection
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()
    print(f"Connected to PostgreSQL: {db_version[0]}")
    
except OperationalError as e:
    print(f"Error connecting to PostgreSQL: {e}")
    connection = None
    cursor = None

Connected to PostgreSQL: PostgreSQL 17.6 (Debian 17.6-1.pgdg12+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit


In [16]:
import pandas as pd

query = """
SELECT
    th.id               AS thesis_id,
    th.title            AS thesis_title,
    st.name             AS student_name,
    th.student_id       AS thesis_author_id,
    p.name              AS advisor_name,
    p.laboratory_id     AS advisor_laboratory_id,
    lab.name            AS advisor_laboratory_name
FROM public.theses         AS th
JOIN public.students       AS st  ON st.id = th.student_id
JOIN public.professors     AS p   ON p.id = th.advisor1_id
JOIN public.laboratories   AS lab ON lab.id = p.laboratory_id;
"""

try:
    # Execute the query
    cursor.execute(query)
    data = cursor.fetchall()
    
    # Get column names
    columns = [desc[0] for desc in cursor.description]
    
    # Convert to pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    print(f"Successfully retrieved {len(df)} thesis records")
    print(f"Columns: {list(df.columns)}")
    
    # Display first few rows
    print("\nFirst 5 rows:")
    print(df.head())
    
except Exception as e:
    print(f"Error executing query: {e}")
    df = None
finally:
    cursor.close()

Successfully retrieved 1335 thesis records
Columns: ['thesis_id', 'thesis_title', 'student_name', 'thesis_author_id', 'advisor_name', 'advisor_laboratory_id', 'advisor_laboratory_name']

First 5 rows:
   thesis_id                                       thesis_title  \
0        873  Diseño de un protocolo para distribuir llaves ...   
1       1090  Autenticación de sensores en una red de área c...   
2       1141  Módulos del núcleo de Linux para el Servicio d...   
3       1266  "A Blind Threshold Signature Scheme for Blockc...   
4       1366  "Protocolo criptográfico para intercambio de i...   

                    student_name  thesis_author_id           advisor_name  \
0               Ruiz Palma Oscar              1106   Gina Gallegos García   
1                El Yazidi  Saad              1327   Gina Gallegos García   
2  Soto Miranda Andrea Viridiana              1375  Eleazar Aguirre Anaya   
3    Reyes Macedo Victor Gabriel              1492   Gina Gallegos García   
4    Delgad

In [17]:
import torch
import gc
import random

def clear_all(model):
    torch.cuda.empty_cache()
    del model
    gc.collect()

def split_data(data):
    # Set random seed for reproducibility
    
    # Create a copy and shuffle the data
    shuffled_data = data.copy()
    random.shuffle(shuffled_data)
    
    # Split the shuffled data
    train_data = shuffled_data[:int(0.8 * len(shuffled_data))]
    test_data = shuffled_data[int(0.8 * len(shuffled_data)):]
    return train_data, test_data



In [18]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [19]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import semantic_search
import re
import torch

models = ["all-MiniLM-L6-v2", "BAAI/bge-m3", "Qwen/Qwen3-Embedding-0.6B"]

# --- Prompt personalizado para Qwen (solo queries) ---
QWEN_CUSTOM_PROMPT = (
    "Instruct: Given a draft thesis title or research topic, retrieve semantically similar thesis titles.\n"
    "Query: "
)

# Detecta dispositivo si no está definido
try:
    device  # noqa
except NameError:
    device = "cuda" if torch.cuda.is_available() else "cpu"


# ----------------- utilidades robustas -----------------
def to_text(record):
    """
    Convierte cualquier estructura (tuple, dict, etc.) en un string razonable.
    - Tuplas/listas: devuelve el primer string no vacío; si no hay, concatena todo.
    - Dicts: intenta 'title', 'thesis_title', 'name', 'text', 'abstract', 'description'.
    - Otros: str(record).
    """
    if record is None:
        return ""
    if isinstance(record, str):
        return record
    if isinstance(record, (list, tuple)):
        for item in record:
            if isinstance(item, str) and item.strip():
                return item
        return " ".join(str(item) for item in record if item is not None)
    if isinstance(record, dict):
        for key in ("title", "thesis_title", "name", "text", "abstract", "description"):
            val = record.get(key)
            if isinstance(val, str) and val.strip():
                return val
        return str(record)
    return str(record)


def clean_text(text):
    text = to_text(text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


# ----------------- split y limpieza -----------------
database, query = split_data(data)

if query is not None:
    query = query[:5]
else:
    query = []

# Normaliza a strings limpios (sin perder el original)
database_texts = [clean_text(x) for x in database]
query_texts = [clean_text(x) for x in query]

for model_name in models:

    if model_name == "Qwen/Qwen3-Embedding-0.6B":
        model = SentenceTransformer(
            "Qwen/Qwen3-Embedding-0.6B",
            model_kwargs={
                "attn_implementation": "flash_attention_2",
                "device_map": "auto",
                "torch_dtype": torch.float16,
            },
            tokenizer_kwargs={"padding_side": "left"},
        )
    else:
        model = SentenceTransformer(model_name, device=device)

    # --------- DOCUMENTOS: sin prompt, normalizados ---------
    database_embeddings = model.encode(
        database_texts,
        show_progress_bar=True,
        device=device,
        normalize_embeddings=True,
    )

    # --------- QUERIES: con prompt custom en Qwen ---------
    if model_name == "Qwen/Qwen3-Embedding-0.6B":
        # Opción A: usando el argumento 'prompt'
        query_embeddings = model.encode(
            query_texts,
            show_progress_bar=True,
            device=device,
            normalize_embeddings=True,
            prompt=QWEN_CUSTOM_PROMPT,  # <<-- clave
        )
        # (Alternativa: concatenar manualmente: [QWEN_CUSTOM_PROMPT + q for q in query_texts])
    else:
        query_embeddings = model.encode(
            query_texts,
            show_progress_bar=True,
            device=device,
            normalize_embeddings=True,
        )

    print(
        f"Generated embeddings for {len(database_texts)} database entries and {len(query_texts)} query entries."
    )

    # --------- Búsqueda semántica ---------
    top_k = 5
    hits = semantic_search(query_embeddings, database_embeddings, top_k=top_k)

    print(f"\n{'='*80}")
    print(f"SEMANTIC SEARCH RESULTS - MODEL: {model_name}")
    print(f"{'='*80}\n")

    for i, query_hits in enumerate(hits):
        print(f"QUERY {i+1}:")
        print(f"Title: {query_texts[i]}")
        print(f"{'-'*60}")
        print(f"TOP {top_k} SIMILAR THESES:")

        for rank, hit in enumerate(query_hits, 1):
            corpus_id = hit["corpus_id"]
            similarity_score = hit["score"]
            similar_thesis = database_texts[corpus_id]
            print(f"\n  Rank {rank} (Similarity: {similarity_score:.4f}):")
            print(f"  Title: {similar_thesis}")

        print(f"\n{'='*80}\n")

    # Liberar
    model = None
    clear_all(model)

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings for 1068 database entries and 5 query entries.

SEMANTIC SEARCH RESULTS - MODEL: all-MiniLM-L6-v2

QUERY 1:
Title: "Detección de indicadores de salud mental en esquizofrenia por medio de sensado pasivo de datos en teléfonos celulares"
------------------------------------------------------------
TOP 5 SIMILAR THESES:

  Rank 1 (Similarity: 0.6382):
  Title: Sistema reporteador para información del servicio telefónico medido (SRSISTEM)

  Rank 2 (Similarity: 0.6295):
  Title: Sistema de asignación de recursos para entidades móviles basado en una infraestructura de telefonía celular

  Rank 3 (Similarity: 0.6084):
  Title: "Detección de salud mental de personas a partir de señales de dispositivos móviles: caso depresión"

  Rank 4 (Similarity: 0.6012):
  Title: "Análisis y detección de aplicaciones coludidas en teléfonos inteligentes con sistema operativo Android"

  Rank 5 (Similarity: 0.5957):
  Title: Servicios de localización sobre una infraestructura de telefonía

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings for 1068 database entries and 5 query entries.

SEMANTIC SEARCH RESULTS - MODEL: BAAI/bge-m3

QUERY 1:
Title: "Detección de indicadores de salud mental en esquizofrenia por medio de sensado pasivo de datos en teléfonos celulares"
------------------------------------------------------------
TOP 5 SIMILAR THESES:

  Rank 1 (Similarity: 0.7224):
  Title: "Detección de salud mental de personas a partir de señales de dispositivos móviles: caso depresión"

  Rank 2 (Similarity: 0.6225):
  Title: Monitoreo y análisis de variables fisiológicas con un enfoque hacia los dispositivos móviles

  Rank 3 (Similarity: 0.5783):
  Title: “Detección temprana de la depresión en textos”

  Rank 4 (Similarity: 0.5778):
  Title: “Estudio del desempeño de sistemas de comunicaciones en semáforos en ciudades inteligentes mediante Li-Fi”

  Rank 5 (Similarity: 0.5749):
  Title: “Detección automática de emociones en transcripciones de terapias psicológicas”


QUERY 2:
Title: "Identificación 

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings for 1068 database entries and 5 query entries.

SEMANTIC SEARCH RESULTS - MODEL: Qwen/Qwen3-Embedding-0.6B

QUERY 1:
Title: "Detección de indicadores de salud mental en esquizofrenia por medio de sensado pasivo de datos en teléfonos celulares"
------------------------------------------------------------
TOP 5 SIMILAR THESES:

  Rank 1 (Similarity: 0.7666):
  Title: "Detección de salud mental de personas a partir de señales de dispositivos móviles: caso depresión"

  Rank 2 (Similarity: 0.6523):
  Title: Herramienta de diálogo automatizada para apoyar el diagnóstico de enfermedades mentales

  Rank 3 (Similarity: 0.6392):
  Title: Sistema de monitoreo preventivo para personas vulnerables mediante mediciones de signos vitales y eventos

  Rank 4 (Similarity: 0.6343):
  Title: Herramienta auxiliar para el diagnóstico de algunos trastornos mentales y su recomendación terapéutica

  Rank 5 (Similarity: 0.6123):
  Title: “Detección temprana de la depresión en textos”


Q

In [20]:
# ──────────────────────────────────────────────────────────────────────────────
# SISTEMA HÍBRIDO  (BM25  ➜  Embeddings)  ·  Comparación de tres modelos
# Solo se modificó / añadió lo estrictamente necesario respecto a tu código.
# ──────────────────────────────────────────────────────────────────────────────
from rank_bm25 import BM25Okapi  # 🔹 BM25 lexical index
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import semantic_search
import numpy as np
import re

# ------------------------------------------------------------------
# 1) Configuración
# ------------------------------------------------------------------
models = ["all-MiniLM-L6-v2", "BAAI/bge-m3", "Qwen/Qwen3-Embedding-0.6B"]
bm25_topN = 50  # nº de candidatos léxicos antes de ranking semántico
final_topK = 5  # nº de tesis que se mostrarán

# ------------------------------------------------------------------
# 2) Preparar data · limpiar textos  · construir índice BM25
# ------------------------------------------------------------------
# database y query provienen de split_data(data)

# Limpiamos todos los títulos - Extraer título (índice 1) de cada tupla primero
database = [clean_text(t[1]) for t in database]
query = [clean_text(t[1]) for t in query]

# Índice BM25 (tokenización muy básica)
tokenized_corpus = [doc.lower().split() for doc in database]
bm25 = BM25Okapi(tokenized_corpus)

# ------------------------------------------------------------------
# 3) Bucle principal · Embeddings + filtro BM25
# ------------------------------------------------------------------
for model_name in models:

    # ----- carga del modelo -----
    if model_name == "Qwen/Qwen3-Embedding-0.6B":
        model = SentenceTransformer(
            model_name,
            model_kwargs={
                "attn_implementation": "flash_attention_2",
                "device_map": "auto",
                "torch_dtype": torch.float16,
            },
            tokenizer_kwargs={"padding_side": "left"},
        )
    else:
        model = SentenceTransformer(model_name)

    print(f"\nUsing model: {model_name}")

    # ----- embeddings precomputados -----
    database_embeddings = model.encode(
        database, show_progress_bar=True, device=device, normalize_embeddings=True
    )
    query_embeddings = model.encode(
        query, show_progress_bar=True, device=device, normalize_embeddings=True
    )

    print(
        f"Generated embeddings for {len(database)} database entries "
        f"and {len(query)} query entries."
    )

    # ----------------------------------------------------------------
    # 4) Procesar cada consulta: BM25 ➜ Embeddings ➜ semantic_search
    # ----------------------------------------------------------------
    print(f"\n{'='*80}")
    print(f"SEMANTIC SEARCH RESULTS (BM25 + {model_name})")
    print(f"{'='*80}\n")

    for idx, (q_text, q_emb) in enumerate(zip(query, query_embeddings), start=1):

        # ---- 4.1  Filtrado BM25 ----
        tokenized_q = q_text.lower().split()
        bm25_scores = bm25.get_scores(tokenized_q)
        cand_ids = np.argsort(bm25_scores)[::-1][:bm25_topN]

        # ---- 4.2  Ranking denso -----
        cand_embs = database_embeddings[cand_ids]
        
        # Convertir a tensores de PyTorch si son arrays de NumPy
        if isinstance(cand_embs, np.ndarray):
            cand_embs = torch.from_numpy(cand_embs)
        if isinstance(q_emb, np.ndarray):
            q_emb = torch.from_numpy(q_emb)
            
        hits = semantic_search([q_emb], cand_embs, top_k=final_topK)[0]

        # ---- 4.3  Mostrar resultados ----
        print(f"QUERY {idx}:")
        print(f"Title: {q_text}")
        print(f"{'-'*60}")
        print("TOP 3 SIMILAR THESES:")

        for rank, hit in enumerate(hits, start=1):
            global_id = cand_ids[hit["corpus_id"]]  # map back to original list
            score = hit["score"]
            print(f"\n  Rank {rank} (Similarity: {score:.4f}):")
            print(f"  Title: {database[global_id]}")

        print(f"\n{'='*80}\n")

    # ----- liberar memoria -----
    model = None
    clear_all(model)


Using model: all-MiniLM-L6-v2


Batches:   0%|          | 0/34 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings for 1068 database entries and 5 query entries.

SEMANTIC SEARCH RESULTS (BM25 + all-MiniLM-L6-v2)

QUERY 1:
Title: "Detección de indicadores de salud mental en esquizofrenia por medio de sensado pasivo de datos en teléfonos celulares"
------------------------------------------------------------
TOP 3 SIMILAR THESES:

  Rank 1 (Similarity: 0.6084):
  Title: "Detección de salud mental de personas a partir de señales de dispositivos móviles: caso depresión"

  Rank 2 (Similarity: 0.5911):
  Title: Detección local de aplicaciones maliciosas en teléfonos inteligentes con sistema operativos Android

  Rank 3 (Similarity: 0.4771):
  Title: Una técnica basada en realidad virtual para el entrenamiento de especialistas de salud

  Rank 4 (Similarity: 0.4622):
  Title: "Detección de patrones de interés en noticias mediante procesamiento masivo"

  Rank 5 (Similarity: 0.4370):
  Title: "Minería de datos para la detección de anomalías en apoyos gubernamentales".


QUERY 2:
Titl

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings for 1068 database entries and 5 query entries.

SEMANTIC SEARCH RESULTS (BM25 + BAAI/bge-m3)

QUERY 1:
Title: "Detección de indicadores de salud mental en esquizofrenia por medio de sensado pasivo de datos en teléfonos celulares"
------------------------------------------------------------
TOP 3 SIMILAR THESES:

  Rank 1 (Similarity: 0.7224):
  Title: "Detección de salud mental de personas a partir de señales de dispositivos móviles: caso depresión"

  Rank 2 (Similarity: 0.5577):
  Title: "Detección de emociones negativas en textos cortos con aprendizaje profundo"

  Rank 3 (Similarity: 0.5535):
  Title: "Detección de enfermedades en la piel por medio de notas clínicas e imágenes, utilizando redes neuronales recurrentes"

  Rank 4 (Similarity: 0.5235):
  Title: "Detección de patrones de interés en noticias mediante procesamiento masivo"

  Rank 5 (Similarity: 0.5130):
  Title: "Minería de datos para la detección de anomalías en apoyos gubernamentales".


QUERY 2:


Batches:   0%|          | 0/34 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings for 1068 database entries and 5 query entries.

SEMANTIC SEARCH RESULTS (BM25 + Qwen/Qwen3-Embedding-0.6B)

QUERY 1:
Title: "Detección de indicadores de salud mental en esquizofrenia por medio de sensado pasivo de datos en teléfonos celulares"
------------------------------------------------------------
TOP 3 SIMILAR THESES:

  Rank 1 (Similarity: 0.8599):
  Title: "Detección de salud mental de personas a partir de señales de dispositivos móviles: caso depresión"

  Rank 2 (Similarity: 0.6836):
  Title: Base de datos de señales EEG estimuladas por imágenes de frutas

  Rank 3 (Similarity: 0.6709):
  Title: Detección local de aplicaciones maliciosas en teléfonos inteligentes con sistema operativos Android

  Rank 4 (Similarity: 0.6362):
  Title: "Detección de enfermedades en la piel por medio de notas clínicas e imágenes, utilizando redes neuronales recurrentes"

  Rank 5 (Similarity: 0.6323):
  Title: Análisis de redes celulares asistidas por drones usando técnicas